In [112]:
import json
import pandas as pd
from yome import Session
from yome.models import *
from yome.util import to_df, report
import re

In [113]:
from sqlalchemy import exists
from sqlalchemy.orm import aliased

# Look for genes that have absolutely no information

In [114]:
session = Session()

# aliases
KnowledgebaseGene2 = aliased(KnowledgebaseGene)
Knowledgebase2 = aliased(Knowledgebase)
KnowledgebaseFeature2 = aliased(KnowledgebaseFeature)

# Genes with no info in EcoCyc
no_info = (
    session.query(Gene.locus_id.label('locus_tag'), 
                  KnowledgebaseGene.id.label('knowledgebase_gene_id'), 
                  KnowledgebaseGene.primary_name.label('ecocyc_primary_name'),
                  Knowledgebase, 
                  KnowledgebaseFeature.feature)
    .join(KnowledgebaseGene)
    .join(Knowledgebase)
    .join(KnowledgebaseFeature)
    .filter(Knowledgebase.name == 'EcoCyc')
    .filter(KnowledgebaseFeature.feature_type == 'summary_html')
    .filter(KnowledgebaseFeature.feature.like('%No information about this%'))
)
# Make a subquery
no_info_sub = no_info.subquery('no_info_sub')

# Genes that do not start with y in EcoCyc and have no information
no_info_not_start_y = (
    no_info.filter(~KnowledgebaseGene.primary_name.like('y%'))
)

# Genes with no info in EcoCyc that are not ranked "high" in another db
no_info_and_not_high = (
    no_info
    .filter(
        ~session.query(KnowledgebaseGene2, Knowledgebase2)
        .filter(KnowledgebaseGene2.gene_id == Gene.id)
        .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
        .filter(KnowledgebaseGene2.annotation_quality == 'high')
        .exists()
    )
)

# Genes with no info in EcoCyc that are ranked "high" in another db
no_info_and_high = (
    no_info
    .filter(
        session.query(KnowledgebaseGene2, Knowledgebase2)
        .filter(KnowledgebaseGene2.gene_id == Gene.id)
        .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
        .filter(KnowledgebaseGene2.annotation_quality == 'high')
        .exists()
    )
)

# Examples of no_info_and_not_high
no_info_and_not_high_examples = (
    session.query(Gene.locus_id,
                  KnowledgebaseGene.id,
                  KnowledgebaseGene2.primary_name,
                  KnowledgebaseGene2.annotation_quality,
                  Knowledgebase2.name,
                  KnowledgebaseFeature2.feature_type,
                  KnowledgebaseFeature2.feature)
    .filter(Gene.locus_id == no_info_sub.c.locus_tag)
    .filter(KnowledgebaseGene.id == no_info_sub.c.knowledgebase_gene_id)
    .join(KnowledgebaseGene2, KnowledgebaseGene2.gene_id == Gene.id)
    .join(Knowledgebase2, Knowledgebase2.id == KnowledgebaseGene2.knowledgebase_id)
    .join(KnowledgebaseFeature2, KnowledgebaseFeature2.knowledgebase_gene_id == KnowledgebaseGene2.id)
    .filter(Knowledgebase2.name != 'EcoCyc')
    .filter(Knowledgebase2.name != 'Y-ome')
)

session.close()

In [115]:
no_info.count(), no_info_not_start_y.count(), no_info_and_not_high.count(), no_info_and_high.count(), no_info_and_not_high_examples.count()

(524, 55, 511, 13, 5640)

In [116]:
df = to_df(no_info_and_not_high_examples, ['locus_tag', 'ig1', 'name', 'annotation_quality', 'kbase', 'feature_type', 'feature']).drop('ig1', axis=1)

In [117]:
df[df.locus_tag == 'b1811']

,locus_tag,name,annotation_quality,kbase,feature_type,feature
2139,b1811,yoaH,low,EcoGene,protein,UPF0181 family protein
2140,b1811,yoaH,low,EcoGene,function,Null
2141,b1811,yoaH,low,EcoGene,description,"UPF0181 family protein, function unknown"
2142,b1811,yoaH,low,EcoGene,comments,Null
2143,b1811,yoaH,low,EcoGene,is_pseudogene,f
4502,b1811,yoaH,low,UniProt,upid,P67338
4503,b1811,yoaH,low,UniProt,protein,UPF0181 protein YoaH
4504,b1811,yoaH,low,UniProt,features,Chain (1)
4505,b1811,yoaH,low,UniProt,annotation_score,1
5056,b1811,yoaH,tbd,RefSeq,ec_numbers,{}


## Filter out phrases for other databases

In [118]:
# TODO This doesn't capture cases like b1811 EcoGene and UniProt don't even have features to check

phrases = [
    ('EcoGene', r'.* family protein$'),
    ('EcoGene', r'.* family\.?$'),
    ('EcoGene', r'Null'),
    ('EcoGene', r'Putative enzyme; Not classified'),
    ('EcoGene', r'Phenotype; Not classified'),
    ('EcoGene', r'Putative factor; Not classified'),
    ('EcoGene', r'.*uncharacterized protein.*'),
    ('EcoGene', r'.*pseudogene.*'),
    ('EcoGene', r'.*function unknown.*'),
    ('EcoGene', r'.*Phage or Prophage Related.*'),
    ('EcoGene', r'^t|f$'),
    ('EcoGene', r'Orf; Not classified'),
    ('UniProt', r'^$'),
    ('UniProt', r'Chain \(1\)'),
    ('UniProt', r'^\d\.0$'),
    ('UniProt', r'^\d$'),
    ('UniProt', r'^Putative uncharacterized'),
    ('UniProt', r'^Putative protein'),
    ('UniProt', r'^Uncharacterized protein'),
    ('UniProt', r'^Protein .{4}$'),
    ('UniProt', r'^UPF\d{4} Protein .{4}$'),
    ('UniProt', r'FUNCTION: Not yet known\.'),
    ('UniProt', r'^t|f$'),
    ('RefSeq', r' family protein$'),
    ('RefSeq', r'^{}$'),
    ('RefSeq', r'^t|f$'),
    ('RefSeq', r'uncharacterized protein'),
    ('RefSeq', r'putative enzyme; Not classified'),
    ('RefSeq', r'putative regulator; Not classified'),
    ('RefSeq', r'orf; Not classified'),
    ('RefSeq', r'putative factor; Not classified'),
]
ignore_feature_types = [
    ('UniProt', 'upid')
]
df_filtered = df
for kbase, reg in phrases:
    df_filtered = df_filtered[~((df_filtered.kbase == kbase) & df_filtered.feature.str.contains(reg, flags=re.IGNORECASE))]
for kbase, feature_type in ignore_feature_types:
    df_filtered = df_filtered[~((df_filtered.kbase == kbase) & (df_filtered.feature_type == feature_type))]

In [137]:
# df_filtered.to_csv('no-ecocyc-info-other-db-annotations.tsv', sep='\t')

In [120]:
# Find no info genes that are not in the filtered list
session = Session()
no_info_not_in_filtered = (
    no_info
    .filter(Gene.locus_id.notin_(df_filtered.loc[:, 'locus_tag']))
)
session.close()

In [121]:
# double check numbers add up
assert(no_info.count() == no_info_not_in_filtered.count() + len(df_filtered.locus_tag.unique()))

## These are the genes that really have no hints at all:

In [130]:
no_info_not_in_filtered.count()

131

In [131]:
no_info_not_in_filtered_df = to_df(no_info_not_in_filtered)

In [132]:
len(no_info_not_in_filtered_df)

131

## List with primary names

In [141]:
no_info_not_in_filtered_df.loc[:, ['locus_tag', 'ecocyc_primary_name']]

,locus_tag,ecocyc_primary_name
0,b0821,ybiU
1,b2181,yejG
2,b3148,yraN
3,b4536,yobH
4,b0326,yahL
5,b1172,ymgG
6,b4567,yjjZ
7,b3354,yheU
8,b4546,ypeB
9,b4537,yecJ
